In [1]:
#Importing packages
from transformers import ViTModel, ViTImageProcessor
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import chromadb
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
#Import library code
import dataloading
import model_functions
import data_vis

d:\Projects\DSC180A-Capstone\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
labels_csv = "camera_data/coronado_hills_binary_11-15-2025.csv"
image_dir = "camera_data/images/"

data = dataloading.get_data(labels_csv=labels_csv, image_dir=image_dir)

In [3]:
train, val, test = dataloading.get_train_val_test(data = data, output_csvs=True)

In [4]:
train_embeddings, train_labels, _, _  = dataloading.load_full_embeddings(train, "q1_train_embeddings")

In [5]:
train_embedding_dataset = dataloading.CustomEmbeddingDataset(train_embeddings, train_labels)
#val_embedding_dataset = dataloading.CustomEmbeddingDataset(embeddings, labels)

train_embedding_dataloader = DataLoader(train_embedding_dataset, batch_size=32, shuffle=True, pin_memory=True)
#val_embedding_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, pin_memory=True)

In [6]:
device = "cuda"

encoder = model_functions.create_encoder()
encoder.to(device)

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTEmbeddingNet(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermediate_act_f

In [7]:
encoder.load_state_dict(torch.load('weights/model_weights_camera_11-17-25.pth', weights_only=True))
encoder.eval();

In [8]:
classification_head = model_functions.ClassificationHead()
classification_head.to(device)

ClassificationHead(
  (head): Sequential(
    (0): Linear(in_features=768, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
  )
)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classification_head.parameters(), lr=1e-4) # Optimize only the new head

In [10]:
num_epochs = 1
for epoch in range(num_epochs):
    classification_head.train() # Set model to training mode
    for batch in train_embedding_dataloader:
        embeddings = batch['embeddings'].to(device).float()
        labels = batch['labels'].to(device).long()

        optimizer.zero_grad()
        outputs = classification_head(embeddings)
        print("Embeddings shape:", embeddings.shape)
        print("Embeddings dtype:", embeddings.dtype)

        print("Outputs shape:", outputs.shape)
        print("Outputs dtype:", outputs.dtype)

        print("Has NaN:", torch.isnan(outputs).any())
        print("Has Inf:", torch.isinf(outputs).any())

        print("Model param dtype:", next(classification_head.parameters()).dtype)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

Embeddings shape: torch.Size([32, 768])
Embeddings dtype: torch.float32
Outputs shape: torch.Size([32, 2])
Outputs dtype: torch.float32
Has NaN: tensor(True, device='cuda:0')
Has Inf: tensor(False, device='cuda:0')
Model param dtype: torch.float32


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
